In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import lightgbm as lgb

In [6]:
# read the data from /input/train.csv
data = pd.read_csv("train.csv")

In [7]:
X = data.drop(['Class', 'id'], axis=1)
y = data['Class']

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## LGBM

In [31]:
model = lgb.LGBMClassifier(n_estimators=512, max_depth=2, random_state=42)

In [32]:
# fit the model
model.fit(X_train, y_train)

LGBMClassifier(max_depth=2, n_estimators=512, random_state=42)

In [33]:
preds = model.predict_proba(X_test)

In [34]:
# log loss
logloss = log_loss(y_test, preds)
print("Log loss: ", logloss)


Log loss:  0.03226194048559742


In [36]:
# make training predictions
train_probs = model.predict_proba(X_train)

# calculate log loss
train_loss = log_loss(y_train, train_probs)
train_loss

0.0269129277027054

In [89]:
# read the data from sample_submission.csv
submission = pd.read_csv("sample_submission.csv")

In [90]:
submission.head()

,id,Class
0,117564,0.5
1,117565,0.5
2,117566,0.5
3,117567,0.5
4,117568,0.5


In [159]:
# Save test predictions to file
submission = pd.DataFrame({'id': X_test.id,
                       'Class': preds})
submission.to_csv('submission.csv', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'id'

In [154]:
type(X_test)

numpy.ndarray

In [37]:
# read the real test data from test.csv
test_data = pd.read_csv("test.csv")

In [38]:
test_data.head()

,id,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,117564,140.046875,54.507800,0.058862,-0.567263,2.337793,14.868335,9.591760,117.988781
1,117565,107.828125,51.578965,0.284368,-0.338430,1.574415,12.501437,11.694968,182.704822
2,117566,135.062500,49.812343,-0.087784,-0.094341,3.576923,21.243336,7.252386,59.021499
3,117567,112.812500,41.926647,0.519921,1.287762,6.669732,29.013153,5.097661,27.105240
4,117568,96.210938,35.322620,0.481286,2.443080,2.218227,17.041064,9.766006,117.131775


In [39]:
X_testtest = test_data.drop(['id'], axis=1)

In [40]:
ID_test = test_data.id

In [42]:
# final fit with entire training set
model.fit(X, y)

LGBMClassifier(max_depth=2, n_estimators=512, random_state=42)

In [43]:
preds = model.predict_proba(X_testtest)

In [44]:
preds

array([[9.99965367e-01, 3.46326290e-05],
       [9.99281118e-01, 7.18881666e-04],
       [9.99925887e-01, 7.41133349e-05],
       ...,
       [9.99945530e-01, 5.44697810e-05],
       [9.26492558e-01, 7.35074420e-02],
       [2.70453651e-02, 9.72954635e-01]])

In [45]:
preds=  preds[:,1]

In [46]:
# Save test predictions to file
submission = pd.DataFrame({'id': ID_test,
                       'Class': preds})
submission.to_csv('submission_lgbm.csv', index=False)